In [7]:
from rdflib import Namespace, Graph, Literal, URIRef
import re
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from buildingmotif.namespaces import bind_prefixes
from buildingmotif.model_builder import TemplateBuilderContext
import os 
import sys 
# %%
# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://")

# create the model w/ a namespace
WBS = Namespace("urn:ex/")
S223 = Namespace("http://data.ashrae.org/standard223#")
wbs = Model.create(WBS)
bind_prefixes(wbs.graph)
wbs.graph.bind("wbs", WBS)
wbs.graph.bind("s223", S223)
wbs.graph.bind("nawi", Namespace("urn:nawi-water-ontology#"))
things = []

In [8]:
sys.path.append('..')
from utils import * # ModelBuilder and vars

In [9]:
# %%
templates = Library.load(directory='../templates')
s223 = Library.load(ontology_graph="../../s223/collections/MODEL_SP223_all-v1.0.ttl")
templates.get_templates()

/home/fletch/miniconda3/lib/python3.12/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))
2025-06-03 11:59:14,157 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/model/core (Name: http://data.ashrae.org/standard223/1.0/model/core). Check this is loaded into BuildingMOTIF
2025-06-03 11:59:14,159 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/currency (Name: http://qudt.org/2.1/vocab/currency). Check this is loaded into BuildingMOTIF
2025-06-03 11:59:14,160 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/vocab/enumeration (Name: http://data.ashrae.org/standard223/1.0/vocab/enumeration). Check this is loaded into BuildingMOTIF
2025-06-03 11:59:14,161 | ro

[Template(_id=1, _name='ph', body=<Graph identifier=6d48d11e-508b-48e3-ab66-cbe67e0590ee (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x7f672eb32ba0>),
 Template(_id=2, _name='pressure', body=<Graph identifier=c3891374-7bbb-4e3d-bd8c-6273f2a68ab4 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x7f672eb32ba0>),
 Template(_id=3, _name='conductivity', body=<Graph identifier=aa300586-acc8-49b0-8e63-23dad2a93290 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x7f672eb32ba0>),
 Template(_id=4, _name='temperature', body=<Graph identifier=fd58ec7f-3b29-47e7-8903-ae5350bf97b7 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x7f672eb32ba0>),
 Template(_id=5, _name='flow-rate', body=<Graph ide

In [16]:
builder = ModelBuilder(ns = WBS, bm_instance= wbs, templates = templates)

In [17]:
# TODO: how to add column 53, "GAC-UVT (%)" between GAC and UV system?
h2o2_dosing_pump = builder.create_equipment('h2o2-tank', 'tank', role = S223['Role-Supply'])
h2o2_dosing_pump_props = builder.add_properties(h2o2_dosing_pump['out'], ['status', 'flow-rate', 'speed-command'], ['145', '94', '68'])
lpuv_bypass_valve = builder.create_equipment('lpuv-bypass-valve', 'valve')
builder.connect_equipment(h2o2_dosing_pump, lpuv_bypass_valve)
lpuv_bypass_outlet = builder.add_connection_point(lpuv_bypass_valve, 'outlet-cp')
# TODO: should low pressure UV be a subclass of standard UV reactor? Right now all 4 are the same class
lpuv_reactor_1 = builder.create_equipment('lpuv-1', 'uv-reactor')
# TODO: column 26 is "GAC-C3 Flow (GPM)", which doesn't seem to match the LPUV reactor...
# lpuv_reactor_1_props = builder.add_properties(lpuv_reactor_1['name'], ['status'], ['26'])
lpuv_reactor_2 = builder.create_equipment('lpuv-2', 'uv-reactor')
lpuv_reactor_2_props = builder.add_properties(lpuv_reactor_2['name'], ['status'], ['141']) #, '107'])
lpuv_reactor_3 = builder.create_equipment('lpuv-3', 'uv-reactor')
# TODO: column 107 is "SCADA Enclosure Door", which doesn't seem to match the LPUV reactor...
# lpuv_reactor_3_props = builder.add_properties(lpuv_reactor_1['name'], ['status'], ['107'])
lpuv_1_to_2 = builder.connect_equipment(lpuv_reactor_1, lpuv_reactor_2)
lpuv_2_to_3 = builder.connect_equipment(lpuv_reactor_2, lpuv_reactor_3)
# TODO: On / Off and LED % do not have a column numbers
uv_led_reactor = builder.create_equipment('uv-led', 'uv-reactor')
builder.connect_equipment(lpuv_bypass_valve, uv_led_reactor, source_cp=lpuv_bypass_outlet)
uv_outlet_valve = builder.create_equipment('uv-out-valve', 'valve')
builder.connect_equipment(lpuv_reactor_3, uv_outlet_valve)
uv_led_outlet = builder.add_connection_point(uv_outlet_valve, 'inlet-cp')
builder.connect_equipment(uv_led_reactor, uv_outlet_valve, target_cp=uv_led_outlet)

EQUIP HAS ALL NEEDED PROPERTIES:  h2o2-tank
EQUIP HAS ALL NEEDED PROPERTIES:  lpuv-bypass-valve
CP HAS ALL NEEDED PROPERTIES:  urn:ex/lpuv-bypass-valve-outlet-cp-2
EQUIP HAS ALL NEEDED PROPERTIES:  lpuv-1
EQUIP HAS ALL NEEDED PROPERTIES:  lpuv-2
EQUIP HAS ALL NEEDED PROPERTIES:  lpuv-3
EQUIP HAS ALL NEEDED PROPERTIES:  uv-led
EQUIP HAS ALL NEEDED PROPERTIES:  uv-out-valve
CP HAS ALL NEEDED PROPERTIES:  urn:ex/uv-out-valve-inlet-cp-2


{'in': rdflib.term.URIRef('urn:ex/uv-led-out'),
 'out': rdflib.term.URIRef('urn:ex/uv-out-valve-inlet-cp-2'),
 'name': rdflib.term.URIRef('urn:ex/conn-uv-led-to-uv-out-valve')}

In [18]:
# TODO: how do we want to represent dosing points?
chlorine_dosing_point = builder.create_equipment('chlorine-dose-point', 'junction', role = S223['Role-Supply'])
builder.connect_equipment(uv_outlet_valve, chlorine_dosing_point)
chlorine_dosing_pump = builder.create_equipment('chlorine-dosing-tank', 'tank', role = S223['Role-Supply'])
chlorine_dosing_pump_props = builder.add_properties(chlorine_dosing_pump['out'], ['flow-rate', 'speed-command'], ['93', '67'])
builder.connect_equipment(chlorine_dosing_pump, chlorine_dosing_point)
# TODO: is S223['Role-Supply'] necessary? Is there a better role than 'tank' for a contact chamber?
chlorine_contactor = builder.create_equipment('chlorine-contactor', 'tank', role = S223['Role-Supply'])
effluent_valve = builder.create_equipment('potable-effluent-valve', 'valve')
effluent_pipe = builder.connect_equipment(chlorine_contactor, effluent_valve)
# TODO: effluent pressure indicator (PI / eff) does not have a column number
effluent_pipe_props = builder.add_properties(effluent_pipe['name'], ['toc-ppm', 'toc-ppm', 'ph', 'turbidity', 'cl2-mgL'], [75, 46, 39, 35, 37])

TemplateNotFound: Name: junction

In [13]:
builder.graph.serialize("disinfect-model.ttl", format="turtle")

<Graph identifier=cc69fdc3-0618-43f7-a53a-0d1d14f4a1cc (<class 'rdflib.graph.Graph'>)>